In [ ]:
from aitemplates import Message, ChatSequence, ChatConversation, create_chat_completion, FunctionPair, FunctionsAvailable
import os
import chromadb
from chromadb.utils import embedding_functions
import openai
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY

if OPENAI_API_KEY is None:
    raise Exception("API key not found in environment variables")

chroma_client = chromadb.Client()

embedder = embedding_functions.OpenAIEmbeddingFunction(
    api_key=OPENAI_API_KEY,
    model_name="text-embedding-ada-002",
)

collection = chroma_client.create_collection(name="", embedding_function=embedder)

In [ ]:
system_prompt=""
system_prompt_msg = Message("system", system_prompt)

description = ""
description_msg = Message("system", description)

user_query=""
user_query_msg = Message("user", user_query)

sequence1 = ChatSequence([system_prompt_msg, description_msg, user_query_msg])

sequence2 = ChatSequence([user_query_msg])
chat = ChatConversation([sequence1, sequence2])

In [ ]:
func_desc = {
        "name": "func_name",
        "description": "",
        "parameters": {
            "type": "object",
            "properties": {
                "property1": {
                    "type": "string",
                    "description": "",
                },
            },
            "required": ["property1"]
        },
    }
    
def func(property1: str) -> float:
    return 0.0
    
# match the description to the function
function_pair1 = FunctionPair(func_desc, func)

# add FunctionsAvailable dataclass for easy access
functions_available = FunctionsAvailable([function_pair1])

completion = create_chat_completion(sequence1, functions=functions_available)

In [ ]:

collection.add(
    documents=[completion],
    metadatas=[{"": ""}],
    ids=[""]
)
completion

In [ ]:
from aitemplates import ApiManager

api_manager = ApiManager()
api_manager.total_cost